In [1]:
import torch
from torch import nn

In [3]:
torch.cuda.set_device(3)

In [4]:
device = torch.device('cuda:3' if torch.cuda.is_available() else "cpu")
print(device)

cuda:3


In [6]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate==0.27.2 #0.20.3  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.18.0 #2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-qjok0_45
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-qjok0_45
  Resolved https://github.com/huggingface/transformers.git to commit e34da3ee3c9d2d628fdbeb60cee45c4f8f32945a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-a42__ghe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-a42__ghe
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running 

### Load the final Checkpoint

In [29]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device,
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf", padding_side='left')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

To load a fine-tuned Lora/Qlora adapter use PeftModel.from_pretrained. output_dir should be something containing an adapter_config.json and adapter_model.bin:

In [30]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "BiCodeReview-llama-FinalRun_f/checkpoint-400/", device_map={'':3})

### Evaluating Code-Llama On The Testing Dataset

In [31]:
import pandas as pd
testD = pd.read_csv('./bi_testingData.csv')

In [32]:
testD.head()

,OriginalCode,ReviewedCode
0,private static Set < String > METHOD_1 ( TYPE_...,private static Set < String > METHOD_1 ( TYPE_...
1,"public TYPE_8 ( TYPE_1 VAR_2 , TYPE_2 VAR_3 ) ...","public TYPE_8 ( TYPE_1 VAR_2 , TYPE_2 VAR_3 ) ..."
2,private void METHOD_1 ( ) { if ( ! VAR_1 . MET...,private void METHOD_1 ( ) { if ( ! VAR_1 . MET...
3,public void METHOD_1 ( ) throws TYPE_1 { Map <...,public void METHOD_1 ( ) throws TYPE_1 { final...
4,private TYPE_1 METHOD_1 ( List < TYPE_1 > VAR_...,private TYPE_1 METHOD_1 ( List < TYPE_1 > VAR_...


In [33]:
testRev = testD['ReviewedCode']
testRev

0       private static Set < String > METHOD_1 ( TYPE_...
1       public TYPE_8 ( TYPE_1 VAR_2 , TYPE_2 VAR_3 ) ...
2       private void METHOD_1 ( ) { if ( ! VAR_1 . MET...
3       public void METHOD_1 ( ) throws TYPE_1 { final...
4       private TYPE_1 METHOD_1 ( List < TYPE_1 > VAR_...
                              ...                        
1714    public TYPE_1 < Optional < TYPE_2 > > METHOD_1...
1715    public void METHOD_1 ( ) throws Throwable { ME...
1716    public static void METHOD_1 ( ) { TYPE_1 . MET...
1717    protected void METHOD_1 ( String VAR_1 ) { TYP...
1718    public static int METHOD_1 ( Object key , TYPE...
Name: ReviewedCode, Length: 1719, dtype: object

In [34]:
sysPrompt = "You are a powerful code reviewing model. Your job is to improve Java code. You are given a Java method code and context regarding what to improve in the code.\n\nYou must output the improved version of the code provided as input based on the review comment provided as the context."
eval_prompt = []
for i in range(len(testD)):
    eval_prompt.append(f"""{sysPrompt}\n### Input:\n{str(testD['OriginalCode'][i])}\n\n### Response:\n""")

In [35]:
print(eval_prompt[0])

You are a powerful code reviewing model. Your job is to improve Java code. You are given a Java method code and context regarding what to improve in the code.

You must output the improved version of the code provided as input based on the review comment provided as the context.
### Input:
private static Set < String > METHOD_1 ( TYPE_1 VAR_1 ) { return VAR_1 . METHOD_2 ( ) . stream ( ) . map ( TYPE_2 : : METHOD_3 ) . filter ( VAR_2 - > ! TYPE_3 . contains ( VAR_2 ) ) . collect ( Collectors . VAR_3 ( TYPE_4 : : new ) ) ; }

### Response:



In [36]:
comp_result = dict()

for j, i in enumerate (eval_prompt):
    model_input = tokenizer(i, return_tensors="pt").to(device)

    model.eval()
    with torch.no_grad():
        ans = tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)
        
        pred = ans.split("###",2)[2][11:]
        t_real = testRev[j]
        comp_result['test_case_'+str(j)] = {'actual':t_real, 'predicted':pred}

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [37]:
for i in comp_result:
    comp_result[i]['actual'] = comp_result[i]['actual'].strip()
    comp_result[i]['predicted'] = comp_result[i]['predicted'].strip()

In [38]:
import json
file = './actual_pred_stripped.json' 
with open(file, 'w') as f: 
    json.dump(comp_result, f)